In [88]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import argparse, os, sys

from __future__ import division, print_function, absolute_import
from tf_utils import load_features_with_deltas_stacking, random_mini_batches

FLAGS = None

train_data, test_data, train_labels, test_labels = load_features_with_deltas_stacking()
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)

tf.logging.set_verbosity(tf.logging.INFO)

In [89]:
#Seeding
seed = 3

# Network Parameters
input_h = train_data.shape[1] #Height
input_w = train_data.shape[2] #Width
num_classes = 2

In [90]:
def sound_net(features, labels, mode):

    predictions = None
    loss = None
    train_op = None
    eval_metric_ops = None
    
    # Input Layer
    x = tf.reshape(features["x"], [-1, input_h, input_w, 1])

    # Convolutional Layer #1
    # input shape [batch_size, input_h, input_w, 1]
    # output shape [batch_size, input_h, input_w, 80]        
    conv1 = tf.layers.conv2d(inputs=x, filters=80, kernel_size=[57, 6], padding="same", 
                             activation=tf.nn.relu, name='conv1')

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # input shape [batch_size, input_h, input_w, 80]
    # output shape [batch_size, input_h/2, input_w/2, 80]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], padding="same", strides=2)
    pool1 = tf.layers.dropout(inputs=pool1, rate=FLAGS.dropout, training=mode == tf.estimator.ModeKeys.TRAIN, name='pool1')

    # Convolutional Layer #2
    # input shape [batch_size, input_h/2, input_w/2, 80]
    # output shape [batch_size, input_h/2, input_w/2, 80]
    conv2 = tf.layers.conv2d(inputs=pool1, filters=80, kernel_size=[1, 3], 
                             padding="same", activation=tf.nn.relu, name='conv2')
    
    # Pooling Layer #2
    # Input Tensor Shape: [batch_size, input_h/2, input_w/2, 80]
    # Output Tensor Shape: [batch_size, input_h/16 input_w/14, 80]    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[8, 7], padding="same", 
                                    strides=[8, 7], name='pool1')

    # Fully connected layer #1
    # Reshape conv2 output to fit fully connected layer input
    pool2_flat = tf.reshape(pool2, [-1, int(input_h/16)*int(input_w/14)*80])    
    dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dense1 = tf.layers.dropout(inputs=dense1, rate=FLAGS.dropout, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Fully connected layer #2
    dense2 = tf.layers.dense(inputs=dense1, units=1024, activation=tf.nn.relu)
    dense2 = tf.layers.dropout(inputs=dense2, rate=FLAGS.dropout, training=mode == tf.estimator.ModeKeys.TRAIN)    
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense2, units=num_classes)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels), name='loss_tensor')   
    correct_prediction = tf.equal(tf.argmax(logits, 1), labels) 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy_tensor')
    tf.summary.scalar('accuracy', accuracy)
        
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, eval_metric_ops=eval_metric_ops)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [91]:
def main(_):

    # Create the Estimator or warm-start the last one
    sound_classifier = tf.estimator.Estimator(
      model_fn=sound_net, model_dir=FLAGS.model_folder)
    
    # Set up logging for predictions
    tensors_to_log = {"loss": "loss_tensor", "accuracy": "accuracy_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors_to_log, every_n_iter=100)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=64,
      num_epochs=None,
      shuffle=True)
    
    sound_classifier.train(
      input_fn=train_input_fn,
      steps=FLAGS.max_steps,
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = sound_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [93]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--max_steps', type=int, default=1000,
                      help='Number of steps to run trainer.')
    parser.add_argument('--learning_rate', type=float, default=0.01,
                      help='Initial learning rate')
    parser.add_argument('--dropout', type=float, default=0.1,
                      help='Keep probability for training dropout.')
    parser.add_argument('--model_folder', type=str, default="/tmp/sound_classifier_model_lr_0_001_dr_0_4_4",
                      help='Folder where the model will be saved.')
    
    FLAGS, unparsed = parser.parse_known_args()    
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/sound_classifier_model_lr_0_001_dr_0_4_4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faacb440d68>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/sound_classifier_model_lr_0_001_dr_0_4_4/model.ckpt.
INFO:tensorflow:loss = 2.0044224, accuracy = 0.484375
INFO:tensorflow:loss = 2.0044224, step = 1
INFO:tensorflow:global_step/sec: 4.97503
INFO:tensorflow:loss = 0.6237169, accuracy = 0.6875 (20.102 sec)
INFO:tensorflow:loss = 0.6237169, step = 

SystemExit: 

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [87]:
tf.reset_default_graph()

In [26]:
FLAGS

Namespace(dropout=0.3, learning_rate=0.1, log_dir='/tmp/tensorflow/sound_classifier/logs/sound_classifier', max_steps=10000)

In [39]:
a = "/tmp/sound_classifier_model_lr_0_001_dr_0_4"
type(a)

str